In [1]:
import xport
import numpy as np
import pandas as pd
import pyodbc
import os
import textwrap

import re
import pyperclip

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_colwidth', 0)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

"""
Connect Pltly to Jupyter notebook
"""

from os.path import isfile, join
import os

import plotly.io as pio
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot, plot_mpl
init_notebook_mode(connected=True)

from plotly.subplots import make_subplots

import plotly.express as px
from sklearn.decomposition import PCA
from sklearn import preprocessing

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
sns.set(style="ticks", font='Times New Roman', font_scale=1.0)
matplotlib.rcParams['font.serif'] = 'Times New Roman'
matplotlib.rcParams['font.family'] = "serif"

from plotly.subplots import make_subplots
import plotly.graph_objects as go

# DO NOT USE SAS7BDAT it didnt properly read 'fped_dr1tot_0910.sas7bdat'. Instead, use 'haven' package in R
# from sas7bdat import SAS7BDAT
# %load_ext autoreload
# %autoreload 1
# %aimport NHANES_dataset_loader
# %aimport FNDDS_base

# Base Functions & Properties

In [2]:
corrections_babak = [
    {'Food code': 2047, 'Main food description': 'Salt', 'novaclass': 2},
    {'Food code': 26100100, 'Main food description': 'Fish, NS as to type, raw', 'novaclass': 1},
    {'Food code': 26115000, 'Main food description': 'Flounder, raw', 'novaclass': 1},
    {'Food code': 26119100, 'Main food description': 'Herring, raw', 'novaclass': 1},
    {'Food code': 26121100, 'Main food description': 'Mackerel, raw', 'novaclass': 1},
    {'Food code': 26125100, 'Main food description': 'Ocean perch, raw', 'novaclass': 1},
    {'Food code': 26313100, 'Main food description': 'Mussels, raw', 'novaclass': 1},
    {'Food code': 63123020, 'Main food description': 'Grapes, American type, slip skin, raw', 'novaclass': 1},
    {'Food code': 27116400, 'Main food description': 'Steak tartare (raw ground beef and egg)', 'novaclass': 3},
]

corrections_babak = pd.DataFrame(corrections_babak)

In [3]:
def load_access_tables(dataset_path, tables):
    if len([x for x in pyodbc.drivers() if x.startswith('Microsoft Access Driver')]) == 0:
        raise Exception(
            'You need to install "Access Data Engine" depending your office X32 or X64 it might become challenging to install it.')
    path, file_name = os.path.split(dataset_path)
    df = {}

    connecion_string = 'DRIVER={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={0};'.format(dataset_path)

    cnxn = pyodbc.connect(connecion_string)

    for table_name in tables:
        path_pickle = "{}/{}.pkl".format(path, table_name)

        if os.path.exists(path_pickle):
            print("Loaded from pickle: {}".format(table_name))
            df[table_name] = pd.read_pickle(path_pickle)
            pass
        else:
            query = "SELECT * FROM {}".format(table_name)

            df[table_name] = pd.read_sql(query, cnxn)

            df[table_name].to_pickle(path_pickle)
            pass

        print("Loaded {} --> Table: {} | Number of rows: {}".format(
            dataset_path, table_name, len(df[table_name])))

        pass

    cnxn.close()

    return df
    pass

## Load FNDDS & SR

In [8]:
# path_USDA_raw_data = 'D:/Dropbox (CCNR)/Foodome Team Folder/Menichetti, Giulia/FoodProcessing/Datasets/FoodProX Core Data/USDA Raw Data/'
path_USDA_raw_data = 'input_data/USDA Raw Data/'

In [9]:
datasets_path = {
#     "FoodData_Central 2020-03-31": "D:/Foodome_datasets/FoodData_Central/2020-03-31/FoodData_Central_2020-03-31.accdb",
#     "FNDDS 2009": "D:/Foodome_datasets/FNDDS/2009_2010/FNDDS2009_2010.mdb",
#     "SR24": "D:/Foodome_datasets/USDA_SR/sr24/sr24.mdb",
#     "SR28": "D:/Foodome_datasets/USDA_SR/sr28/sr28.accdb"
    "Flavonoid 2007-2010": path_USDA_raw_data + '/FNDDS/2007_2010 Flavonoid/Flavonoid_Database_0710.mdb'
}

# https://www.ars.usda.gov/northeast-area/beltsville-md-bhnrc/beltsville-human-nutrition-research-center/methods-and-application-of-food-composition-laboratory/mafcl-site-pages/sr11-sr28/
for sr_version in range(20, 29):
#     path = 'D:/Foodome_datasets/USDA_SR/sr{}/sr{}'.format(
#         sr_version,sr_version
#     )
    path = '{}/USDA_SR/sr{}/sr{}'.format(
        path_USDA_raw_data, sr_version,sr_version
    )
    
    if os.path.exists(path + '.accdb'): 
        path = path + '.accdb'
    elif os.path.exists(path + '.mdb'):
        path = path + '.mdb'
    else:
        raise Exception(f'Cant find path for SR {sr_version}')
    
    datasets_path[f'SR {sr_version}'] = path
    pass

# We dont have FNDDS for 1999-2000 version
for year_fndds in np.arange(2001, 2018, 2):
    datasets_path[f'FNDDS {year_fndds}'] = '{}/FNDDS/{}_{}/FNDDS{}_{}.mdb'.format(
        path_USDA_raw_data,
        year_fndds, year_fndds+1,
        year_fndds, year_fndds+1
    )
    pass


FNDDS = {}

for key, path in datasets_path.items():
    if key.startswith('FNDDS') is False:
        continue
    year = int(key.split(' ')[1])
    
    if year != 2009:
        continue
    
    print('--------------------')
    print (datasets_path[f'FNDDS {year}'])
    
    if year < 2015:
        print(datasets_path[f'FNDDS {year}'])
        FNDDS[year] = load_access_tables(dataset_path=datasets_path[f'FNDDS {year}'],
                                       tables=[
                                           "MainFoodDesc", "FNDDSSRLinks", "NutDesc", 'FNDDSNutVal'
                                          ])
        FNDDS[year]['MainFoodDesc'].rename(columns={'Food Code': 'Food code'}, inplace=True)
        FNDDS[year]['FNDDSSRLinks'].rename(columns={'Food Code': 'Food code'}, inplace=True)
        
    else:
        FNDDS[year] = load_access_tables(dataset_path=datasets_path[f'FNDDS {year}'],
                                       tables=[
                                           "MainFoodDesc",
                                           "FNDDSIngred"
                                          ])        
        pass

print('-----------------Flavonoid---------------')
FNDDS['Flavonoid 2007-2010'] = load_access_tables(dataset_path=datasets_path['Flavonoid 2007-2010'],
                                       tables=[
                                           "FlavDesc"
                                          ])

--------------------
input_data/USDA Raw Data//FNDDS/2009_2010/FNDDS2009_2010.mdb
input_data/USDA Raw Data//FNDDS/2009_2010/FNDDS2009_2010.mdb
Loaded from pickle: MainFoodDesc
Loaded input_data/USDA Raw Data//FNDDS/2009_2010/FNDDS2009_2010.mdb --> Table: MainFoodDesc | Number of rows: 7253
Loaded from pickle: FNDDSSRLinks
Loaded input_data/USDA Raw Data//FNDDS/2009_2010/FNDDS2009_2010.mdb --> Table: FNDDSSRLinks | Number of rows: 28055
Loaded from pickle: NutDesc
Loaded input_data/USDA Raw Data//FNDDS/2009_2010/FNDDS2009_2010.mdb --> Table: NutDesc | Number of rows: 65
Loaded from pickle: FNDDSNutVal
Loaded input_data/USDA Raw Data//FNDDS/2009_2010/FNDDS2009_2010.mdb --> Table: FNDDSNutVal | Number of rows: 471445
-----------------Flavonoid---------------
Loaded from pickle: FlavDesc
Loaded input_data/USDA Raw Data//FNDDS/2007_2010 Flavonoid/Flavonoid_Database_0710.mdb --> Table: FlavDesc | Number of rows: 37


# Nutrient Panels

In [6]:
path_FNDDS_processing_score = 'D:/Dropbox (CCNR)/Foodome Team Folder/Menichetti, Giulia/FoodProcessing/Datasets/FNDDS Processing Score/'

# FNDDS_2009-10_Training_Data.csv
FNDDS_2009_99_panel = pd.read_excel(path_FNDDS_processing_score + 'FNDDS_2009_2010 99 Panel Nutrition Facts with Prosessing Score.xlsx')

FNDDS_2009_99_panel.columns = [str(c) for c in FNDDS_2009_99_panel.columns]

# FNDDS_2009_99_panel = FNDDS_2009_99_panel.drop(columns=['pythonlabel_mod', 'Total Vitamin A'])

FNDDS_2009_99_panel = FNDDS_2009_99_panel.rename(columns=
{
    'class': 'predicted_novaclass',
    '04:00:00': '4:0',
    '06:00:00': '6:0',
    '08:00:00': '8:0',
    '10:00:00': '10:0',
    '12:00:00': '12:0',
    '14:00:00': '14:0',
    '16:00:00': '16:0',
    '18:00:00': '18:0',
    '18:01:00': '18:1',
    '18:02:00': '18:2' ,
    '18:03:00': '18:3',
    '20:04:00': '20:4',
    '22:6 n-3': '22:6 n-3',
    '16:01:00': '16:1',
    '18:04:00': '18:4',
    '20:01:00': '20:1',
    '20:5 n-3': '20:5 n-3',
    '22:01:00': '22:1',
    '22:5 n-3': '22:5 n-3',
})

FNDDS_2009_99_panel['predicted_novaclass'] = FNDDS_2009_99_panel['predicted_novaclass'] + 1

len(FNDDS_2009_99_panel)

FNDDS_2009_99_panel

,Protein,Total Fat,Carbohydrate,Alcohol,Water,Caffeine,Theobromine,"Sugars, total","Fiber, total dietary",Calcium,Iron,Magnesium,Phosphorus,Potassium,Sodium,Zinc,Copper,Selenium,Retinol,"Carotene, beta","Carotene, alpha",Vitamin E (alpha-tocopherol),Vitamin D (D2 + D3),"Cryptoxanthin, beta",Lycopene,Lutein + zeaxanthin,Vitamin C,Thiamin,Riboflavin,Niacin,Vitamin B-6,"Folate, total",Vitamin B-12,"Choline, total",Vitamin K (phylloquinone),Folic acid,"Folate, food","Vitamin E, added","Vitamin B-12, added",Cholesterol,"Fatty acids, total saturated",4:0,6:0,8:0,10:0,12:0,14:0,16:0,18:0,18:1,18:2,18:3,20:4,22:6 n-3,16:1,18:4,20:1,20:5 n-3,22:1,22:5 n-3,"Fatty acids, total monounsaturated","Fatty acids, total polyunsaturated",Daidzein,Genistein,Glycitein,Cyanidin,Petunidin,Delphinidin,Malvidin,Pelargonidin,Peonidin,(+)-Catechin,(-)-Epigallocatechin,(-)-Epicatechin,(-)-Epicatechin 3-gallate,(-)-Epigallocatechin 3-gallate,Theaflavin,Thearubigins,Eriodictyol,Hesperetin,Naringenin,Apigenin,Luteolin,Isorhamnetin,Kaempferol,Myricetin,Quercetin,"Theaflavin-3,3'-digallate",Theaflavin-3'-gallate,Theaflavin-3-gallate,(+)-Gallocatechin,Total flavonoids,Total anthocyanidins,Total catechins (monomeric flavan-3-ols only),Total flavan-3-ols,Total flavanones,Total flavones,Total flavonols,Total isoflavones,Food_code,Main_food_description,catnumb,catname,novaclass,predicted_novaclass,p1,p2,p3,p4,processing score
0,0.00,0.00,0.00,0.0,0.20,0.0,0.0,0.00,0.0,0.024,0.00033,0.001,0.000,0.008,38.758,0.00010,0.000030,1.000000e-07,0.000000,0.000000,0.0,0.00000,0.000000e+00,0.0,0.0,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.0000,0.000000e+00,0.000000,0.000000,0.00000,0.000000e+00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2047,"Salt, table",NaN,addition1516,2,3,0.015000,0.902694,0.007778,0.074528,0.529764
1,1.03,4.38,6.89,0.0,87.50,0.0,0.0,6.89,0.0,0.032,0.00003,0.003,0.014,0.051,0.017,0.00017,0.000052,1.800000e-06,0.000060,0.000007,0.0,0.00008,1.000000e-07,0.0,0.0,0.0,0.0050,0.000014,0.000036,0.000177,0.000011,0.000005,5.000000e-08,0.0160,3.000000e-07,0.000000,0.000005,0.00000,0.000000e+00,0.014,2.009,0.000,0.000,0.000,0.063,0.256,0.321,0.919,0.293,1.475,0.374,0.052,0.026,0.0,0.129,0.0,0.040,0.0,0.0,0.0,1.658,0.497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11000000,"Milk, human",9602.0,Human milk,1,2,0.771343,0.016667,0.047593,0.164398,0.196528
2,3.28,1.91,4.85,0.0,89.25,0.0,0.0,5.08,0.0,0.119,0.00003,0.011,0.091,0.142,0.044,0.00043,0.000014,3.100000e-06,0.000053,0.000004,0.0,0.00004,1.200000e-06,0.0,0.0,0.0,0.0001,0.000040,0.000180,0.000092,0.000037,0.000005,4.900000e-07,0.0158,2.000000e-07,0.000000,0.000005,0.00000,0.000000e+00,0.007,1.154,0.057,0.041,0.033,0.046,0.049,0.173,0.513,0.225,0.483,0.065,0.027,0.000,0.0,0.013,0.0,0.001,0.0,0.0,0.0,0.507,0.094,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11100000,"Milk, NFS",1004.0,"Milk, reduced fat",1,2,0.998333,0.000000,0.000000,0.001667,0.001667
3,3.15,3.25,4.80,0.0,88.13,0.0,0.0,5.05,0.0,0.113,0.00003,0.010,0.084,0.132,0.043,0.00037,0.000025,3.700000e-06,0.000045,0.000007,0.0,0.00007,1.300000e-06,0.0,0.0,0.0,0.0000,0.000046,0.000169,0.000089,0.000036,0.000005,4.500000e-07,0.0143,3.000000e-07,0.000000,0.000005,0.00000,0.000000e+00,0.010,1.865,0.075,0.075,0.075,0.075,0.077,0.297,0.829,0.365,0.812,0.120,0.075,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.812,0.195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11111000,"Milk, cow's, fluid, whole",1002.0,"Mil

In [7]:
"""Check all corrections are applied (G corrections and Babak on raw items + salt)"""
# tmp_df = pd.read_csv('D:\Dropbox (CCNR)\Foodome Team Folder\Ravandi, Babak FDB\FoodomeDev\data\FNDDS\FNDDS_2009-10_Training_data_recom_Giulia_Raw_pythonlabel_mod.csv')

tmp_df = pd.read_csv('Training Dataset NOVA/FNDDS_2009-10_Training_data_recom_Giulia_Raw_pythonlabel_mod double check.csv')

print('len(tmp_df):', len(tmp_df))

tmp_df2 = pd.merge(
    FNDDS_2009_99_panel[['Food_code','novaclass','Main_food_description']],
    tmp_df[['Food_code','novaclass','Main_food_description']], 
    on='Food_code'
)

print('Number of foods in training dataset (have manual NOVA class assigned either by NOVA people or us):',
     sum(tmp_df['novaclass'].isin([1,2,3,4])))

tmp_df2[tmp_df2['novaclass_x'] != tmp_df2['novaclass_y']]

len(tmp_df): 7254
Number of foods in training dataset (have manual NOVA class assigned either by NOVA people or us): 2971


,Food_code,novaclass_x,Main_food_description_x,novaclass_y,Main_food_description_y


In [8]:
nutrients_FNDDS = [str(c) for c in FNDDS_2009_99_panel.loc[:,'Protein':'Total isoflavones'].columns]

NutDesc_FNDDS = FNDDS[2009]['NutDesc'][['Nutrient code', 'Nutrient description', 'Tagname']]
NutDesc_FNDDS['from flav 2007-2010 db'] = 0
NutDesc_sel_FNDDS = NutDesc_FNDDS[NutDesc_FNDDS['Nutrient description'].isin(nutrients_FNDDS)]

NutDesc_flav = (
    FNDDS['Flavonoid 2007-2010']['FlavDesc']
    [['Nutrient code', 'Flavonoid description', 'Tagname', 'Flavonoid class']]
    .rename(columns={'Flavonoid description': 'Nutrient description'})
)
NutDesc_flav['from flav 2007-2010 db'] = 1
NutDesc_sel_flav = NutDesc_flav[NutDesc_flav['Nutrient description'].isin(nutrients_FNDDS)]

# len(NutDesc_sel_FNDDS) + len(NutDesc_sel_flav)

panel_99_nuts = pd.concat([NutDesc_sel_FNDDS, NutDesc_sel_flav]).reset_index(drop=True)

if False:
    panel_99_nuts.to_csv(path_FNDDS_processing_score + 'Panel 99 nutrient codes and descriptions.csv')

if len(panel_99_nuts) != 99:
    raise Exception('MUST BE 99')

panel_99_nuts

<ipython-input-8-af608d8be85e>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Nutrient code,Nutrient description,Tagname,from flav 2007-2010 db,Flavonoid class
0,203,Protein,PROCNT,0,NaN
1,204,Total Fat,FAT,0,NaN
2,205,Carbohydrate,CHOCDF,0,NaN
3,221,Alcohol,ALC,0,NaN
4,255,Water,WATER,0,NaN
5,262,Caffeine,CAFFN,0,NaN
6,263,Theobromine,THEBRN,0,NaN
7,269,"Sugars, total",SUGAR,0,NaN
8,291,"Fiber, total dietary",FIBTG,0,NaN
9,301,Calcium,CA,0,NaN


In [9]:
FNDDS_2009_99_panel['processing score'] = (1 - FNDDS_2009_99_panel['p1'] + FNDDS_2009_99_panel['p4']) / 2

FNDDS_2009_99_panel.loc[:, 'Protein':'Total isoflavones'] = (
    FNDDS_2009_99_panel.loc[:, 'Protein':'Total isoflavones']
    .replace(-20, np.inf)
    .apply(np.exp)
    .replace(np.inf, 0)
)

FNDDS_2009_99_panel

,Protein,Total Fat,Carbohydrate,Alcohol,Water,Caffeine,Theobromine,"Sugars, total","Fiber, total dietary",Calcium,Iron,Magnesium,Phosphorus,Potassium,Sodium,Zinc,Copper,Selenium,Retinol,"Carotene, beta","Carotene, alpha",Vitamin E (alpha-tocopherol),Vitamin D (D2 + D3),"Cryptoxanthin, beta",Lycopene,Lutein + zeaxanthin,Vitamin C,Thiamin,Riboflavin,Niacin,Vitamin B-6,"Folate, total",Vitamin B-12,"Choline, total",Vitamin K (phylloquinone),Folic acid,"Folate, food","Vitamin E, added","Vitamin B-12, added",Cholesterol,"Fatty acids, total saturated",4:0,6:0,8:0,10:0,12:0,14:0,16:0,18:0,18:1,18:2,18:3,20:4,22:6 n-3,16:1,18:4,20:1,20:5 n-3,22:1,22:5 n-3,"Fatty acids, total monounsaturated","Fatty acids, total polyunsaturated",Daidzein,Genistein,Glycitein,Cyanidin,Petunidin,Delphinidin,Malvidin,Pelargonidin,Peonidin,(+)-Catechin,(-)-Epigallocatechin,(-)-Epicatechin,(-)-Epicatechin 3-gallate,(-)-Epigallocatechin 3-gallate,Theaflavin,Thearubigins,Eriodictyol,Hesperetin,Naringenin,Apigenin,Luteolin,Isorhamnetin,Kaempferol,Myricetin,Quercetin,"Theaflavin-3,3'-digallate",Theaflavin-3'-gallate,Theaflavin-3-gallate,(+)-Gallocatechin,Total flavonoids,Total anthocyanidins,Total catechins (monomeric flavan-3-ols only),Total flavan-3-ols,Total flavanones,Total flavones,Total flavonols,Total isoflavones,Food_code,Main_food_description,catnumb,catname,novaclass,predicted_novaclass,p1,p2,p3,p4,processing score
0,1.000000,1.000000,1.000000,1.0,1.221403e+00,1.0,1.0,1.000000,1.0,1.024290,1.00033,1.001001,1.000000,1.008032,6.798068e+16,1.00010,1.000030,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.0,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2047,"Salt, table",NaN,addition1516,2,3,0.015000,0.902694,0.007778,0.074528,0.529764
1,2.801066,79.838033,982.401417,1.0,1.001768e+38,1.0,1.0,982.401417,1.0,1.032518,1.00003,1.003005,1.014098,1.052323,1.017145e+00,1.00017,1.000052,1.000002,1.000060,1.000007,1.0,1.000080,1.000000,1.0,1.0,1.0,1.005013,1.000014,1.000036,1.000177,1.000011,1.000005,1.0,1.016129,1.0,1.000000,1.000005,1.000000,1.0,1.014098,7.455858,1.000000,1.000000,1.000000,1.065027,1.291753,1.378506,2.506782,1.340443,4.371036,1.453537,1.053376,1.026341,1.0,1.137690,1.0,1.040811,1.0,1.0,1.0,5.248803,1.643783,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,11000000,"Milk, human",9602.0,Human milk,1,2,0.771343,0.016667,0.047593,0.164398,0.196528
2,26.575773,6.753089,127.740390,1.0,5.764777e+38,1.0,1.0,160.774056,1.0,1.126370,1.00003,1.011061,1.095269,1.152577,1.044982e+00,1.00043,1.000014,1.000003,1.000053,1.000004,1.0,1.000040,1.000001,1.0,1.0,1.0,1.000100,1.000040,1.000180,1.000092,1.000037,1.000005,1.0,1.015925,1.0,1.000000,1.000005,1.000000,1.0,1.007025,3.170851,1.058656,1.041852,1.033551,1.047074,1.050220,1.188866,1.670295,1.252323,1.620930,1.067159,1.027368,1.000000,1.0,1.013085,1.0,1.001001,1.0,1.0,1.0,1.660303,1.098560,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,11100000,"Milk, NFS",1004.0,"Milk, reduced fat",1,2,0.998333,0.000000,0.000000,0.001667,0.001667
3,23.336065,25.790340,121.510418,1.0,1.880930e+38,1.0,1.0,156.022464,1.0,1.119632,1.00003,1.010050,1.087629,1.141108,1.043938e+00,1.00037,1.000025,1.000004,1.000045,1.000007,1.0,1.000070,1.000001,1.0,1.0,1.0,1.000000,1.000046,1.000169,1.000089,1.000036,1.000005,1.0,1.014403,1.0,1.000000,1.000005,1.000000,1.0,1.010050,6.455936,1.077884,1.077884,1.077884,1.077884,1.080042,1.345815,2.291027,1.440514,2.25

In [10]:
if False:
    FNDDS_2009_99_panel.to_excel(path_FNDDS_processing_score + 'FNDDS_2009_2010 99 Panel Nutrition Facts with Prosessing Score.xlsx')

In [11]:
FNDDS_2009_99_panel['predicted_novaclass'].value_counts()
FNDDS_2009_99_panel['novaclass'].value_counts()

12    3559
4     2112
0     724 
3     466 
1     339 
2     54  
Name: novaclass, dtype: int64

In [12]:
tmp = FNDDS[2009]['FNDDSNutVal']
tmp[(tmp['Food code'] == 11111100) & (tmp['Nutrient code'].isin([203, 205, 255]))]

,Food code,Nutrient code,Start date,End date,Nutrient value
195,11111100,203,2009-01-01,2010-12-31,3.10
197,11111100,205,2009-01-01,2010-12-31,4.46
200,11111100,255,2009-01-01,2010-12-31,88.20


# Training Data Given to Giulia

In [7]:
xls = pd.ExcelFile(
    'input_data/Training Data Original Given by NOVA Researchers - Corrections by Giulia Babak FNDDS 2009-10.xls',
)

train_cor = {}

for sheet_name in xls.sheet_names:
    print('Loading sheet: ', sheet_name)
    train_cor[sheet_name] = pd.read_excel(xls, sheet_name)

Loading sheet:  Given to Giulia from NOVA Peopl
Loading sheet:  Giulia Corrections from Matlab
Loading sheet:  vals counts
Loading sheet:  NOVA Subgroups
Loading sheet:  Recom Raws
Loading sheet:  Preds Giulia Corrections
Loading sheet:  RFFNDDSpredS_cleaned_withsalt
Loading sheet:  matlab mainfooddesc5 tabel
Loading sheet:  Recom Beans
Loading sheet:  Recome Home Recipe 
Loading sheet:  Recom Other


In [8]:
N_orig = train_cor['Given to Giulia from NOVA Peopl']
N_orig

,dr12ifdcd,Main_food_description,SR_code,SR_description,SR_code_t,SR_description_t,fc_or_sr,FC_nova_group,FC_nova_subg,SR_nova_group,SR_nova_subg
0,11000000,"Milk, human",1107,"Milk, human, mature, fluid",1107,"Milk, human, mature, fluid",f,1.0,9.0,1,9
1,11100000,"Milk, NFS",1077,"Milk, whole, 3.25% milkfat, with added vitamin D",1077,"Milk, whole, 3.25% milkfat, with added vitamin D",f,1.0,9.0,1,9
2,11100000,"Milk, NFS",1079,"Milk, reduced fat, fluid, 2% milkfat, with added vitamin A and vitamin D",1079,"Milk, reduced fat, fluid, 2% milkfat, with added vitamin A and vitamin D",f,1.0,9.0,1,9
3,11100000,"Milk, NFS",1082,"Milk, lowfat, fluid, 1% milkfat, with added vitamin A and vitamin D",1082,"Milk, lowfat, fluid, 1% milkfat, with added vitamin A and vitamin D",f,1.0,9.0,1,9
4,11100000,"Milk, NFS",1085,"Milk, nonfat, fluid, with added vitamin A and vitamin D (fat free or skim)",1085,"Milk, nonfat, fluid, with added vitamin A and vitamin D (fat free or skim)",f,1.0,9.0,1,9
...,...,...,...,...,...,...,...,...,...,...,...
35055,94100200,"Water, bottled, sweetened, with low or no calorie sweetener",14604,"Water, non-carbonated, bottles, natural fruit flavors, sweetened with low calorie sweetener",14604,"Water, non-carbonated, bottles, natural fruit flavors, sweetened with low calorie sweetener",f,4.0,39.0,4,39
35056,94100300,"Water, fruit flavored, sweetened, with high fructose corn syrup and low calorie sweetener",14637,"Water, with corn syrup and/or sugar and low calorie sweetener, fruit flavored",14637,"Water, with corn syrup and/or sugar and low calorie sweetener, fruit flavored",f,4.0,39.0,4,39
35057,94210100,Propel Water,14462,"QUAKER OATS, PROPEL Fitness Water, fruit-flavored, non-carbonated",14462,"QUAKER OATS, PROPEL Fitness Water, fruit-flavored, non-carbonated",f,4.0,39.0,4,39
35058,94210200,Glaceau Water,14605,"Water with added vitamins and minerals, bottles, sweetened, assorted fruit flavors",14605,"Water with added vitamins and minerals, bottles, sweetened, assorted fruit flavors",f,4.0,39.0,4,39


In [9]:
N_orig = train_cor['Given to Giulia from NOVA Peopl']

babak_recom = train_cor['Recom Raws']
# babak_recom = raw_babak[raw_babak['Initial Raw Selection'] == 1]
raw_babak = babak_recom[babak_recom['Initial Raw Selection'] == 1]

RFFNDDSpredS_cleaned_withsalt = train_cor['RFFNDDSpredS_cleaned_withsalt']
matlab_mainfooddesc5_tabel = train_cor['matlab mainfooddesc5 tabel']

preds_giulia_corrections = train_cor['Preds Giulia Corrections']

food_items_classified_by_NOVA = (
    N_orig[N_orig['fc_or_sr'] == 'f'][['dr12ifdcd', 'FC_nova_group']]
    .drop_duplicates()
    .rename(columns={'dr12ifdcd': 'Food_code'})
) # 2484 items

"""Two Ways to Get Giulia's Corrections"""

giulia_coorections_from_matlab_mainfooddesc5_tabel = pd.merge(matlab_mainfooddesc5_tabel, food_items_classified_by_NOVA, on='Food_code', how="left")
giulia_coorections_from_matlab_mainfooddesc5_tabel = giulia_coorections_from_matlab_mainfooddesc5_tabel[
    (giulia_coorections_from_matlab_mainfooddesc5_tabel["FC_nova_group"] != giulia_coorections_from_matlab_mainfooddesc5_tabel['novaclass']) &
    (giulia_coorections_from_matlab_mainfooddesc5_tabel['novaclassA'].isin([1,2,3,4]))]

giulia_coorections_from_RFFNDDSpredS_cleaned_withsalt = RFFNDDSpredS_cleaned_withsalt[
    (~RFFNDDSpredS_cleaned_withsalt["Food_code"].isin(food_items_classified_by_NOVA['Food_code'])) &
    (RFFNDDSpredS_cleaned_withsalt["novaclass"].isin([1,2,3,4]))
] 
""""""

#########################!FNDDS foods ingreds/FNDDS_food_ingreds_2009_NOVA_corrected.xlsx


recommendations_all = pd.concat([
    giulia_coorections_from_RFFNDDSpredS_cleaned_withsalt[['Food_code', 'Main_food_description', 'novaclass', 'catname']],
    raw_babak[['Food_code', 'Main_food_description', 'NOVA Recommendation', 'catname']].rename(columns={'NOVA Recommendation': 'novaclass'})
])

total_num_food_items = len(N_orig['dr12ifdcd'].unique())

all_training_data = pd.concat([
    recommendations_all[['Food_code', 'novaclass']],
    food_items_classified_by_NOVA.rename(columns={'FC_nova_group':'novaclass'})[['Food_code', 'novaclass']]
])

print(
"""num_food_items_classified_by_NOVA: {}
Total in FNDDS 2009-10: {}
Corrections by Babak (raw): {}
Corrections by Giulia from her matlab code: {}
Dont Forget Salt!!: +1 correction
Total corrections: 478 + 1 + 8 = 487 (our corrections) + 2484 = 2971[=={}] (total training size)
""".format(len(food_items_classified_by_NOVA), 
           total_num_food_items, 
           len(raw_babak),
           len(giulia_coorections_from_matlab_mainfooddesc5_tabel), 
           len(all_training_data)
          ))

num_food_items_classified_by_NOVA: 2484
Total in FNDDS 2009-10: 7253
Corrections by Babak (raw): 8
Corrections by Giulia from her matlab code: 478
Dont Forget Salt!!: +1 correction
Total corrections: 478 + 1 + 8 = 487 (our corrections) + 2484 = 2971[==2971] (total training size)



In [10]:
478 +1 + 8 + 2484
print(recommendations_all['novaclass'].value_counts())

print(food_items_classified_by_NOVA['FC_nova_group'].value_counts())


# recommendations_all

4.0    239
3.0    133
1.0    111
2.0    4  
Name: novaclass, dtype: int64
4.0    1873
3.0    333 
1.0    228 
2.0    50  
Name: FC_nova_group, dtype: int64


In [11]:
# https://datatofish.com/pie-chart-matplotlib/
def pie_nova_frac(nova_df, title):
    print(title)
    nova_df["NOVA"] = "NOVA " + nova_df["novaclass"].astype(int).astype(str)
    
    df = pd.DataFrame(
        nova_df.groupby("NOVA").size(), 
        columns = ["Count"]).reset_index().sort_values(by="NOVA")
    
    Tasks = df["Count"].values

    my_labels = ['NOVA 1', 'NOVA 2', 'NOVA 3', 'NOVA 4']
    
#     plt.rcParams.update({'font.size': 25})
    SMALL_SIZE = 32
    MEDIUM_SIZE = 10
    BIGGER_SIZE = 12

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
    
    
    fig = plt.figure(figsize=(10, 10), dpi=600)
    
#     plt.pie(Tasks,labels=my_labels,autopct='%1.1f%%')
    p, tx, autotexts = plt.pie(
        Tasks,labels = my_labels, autopct='%1.1f%%', colors=['#66c2a5', '#fc8d62', '#8da0cb', '#e78ac3'])
    
    for i, a in enumerate(autotexts):
        a.set_text("{}% ({})".format(
            round((Tasks[i] / sum(Tasks)) * 100, 1),
               Tasks[i]))
    
#     plt.title(title)
    plt.axis('equal')
    plt.tight_layout()
#     fig.set_size_inches(18.5, 10.5)
    
    plt.savefig('Plots Food Processing/pie_{}.png'.format(title), bbox_inches="tight")
    plt.show()
    return plt

path_final_export = 'D:/FProXFinal/'

pie_nova_frac(food_items_classified_by_NOVA.rename(
    columns={"FC_nova_group": "manual novaclass"}), 
              title='Manual Classification {}'.format(len(food_items_classified_by_NOVA)))
food_items_classified_by_NOVA.rename(columns={"FC_nova_group": "novaclass"}).to_csv(path_final_export + 'FigS2/FigS2_A.csv')

# pie_nova_frac(
#     giulia_coorections_from_matlab_mainfooddesc5_tabel[['novaclassA']].rename(columns={"novaclassA": "manual novaclass"}), 
#     title='Giulia Corrections {}'.format(len(giulia_coorections_from_matlab_mainfooddesc5_tabel))
# )
# giulia_coorections_from_matlab_mainfooddesc5_tabel[['Food_code', 'novaclassA']].rename(columns={"novaclassA": "manual novaclass"}).to_csv(path_final_export + 'FigS2/FigS2_B.csv')

# pie_nova_frac(
#     all_training_data, 
#     title='All Training Data {}'.format(len(all_training_data))
# )
# all_training_data.to_csv(path_final_export + 'FigS2/FigS2_C.csv')


Manual Classification 2484


KeyError: 'novaclass'

In [15]:
path_final_export

'D:/FProXFinal/'

In [ ]:
giulia_coorections_from_matlab_mainfooddesc5_tabel[
    giulia_coorections_from_matlab_mainfooddesc5_tabel['novaclassA'] == 3
]

In [ ]:
recommendations_all[recommendations_all["novaclass"] == 4]

In [ ]:
babak_recom

In [ ]:
q = preds_giulia_corrections[preds_giulia_corrections['novaclass'].isin([1,2,3,4])]
q = RFFNDDSpredS_cleaned_withsalt[RFFNDDSpredS_cleaned_withsalt['novaclass'].isin([1,2,3,4])]


print(len(
    q[q['Food_code'].isin(giulia_recom["Food_code"])]
))

print(len(
    q[q['Food_code'].isin(food_items_classified_by_NOVA)]
))

q2 = q[~(q['Food_code'].isin(food_items_classified_by_NOVA) | q['Food_code'].isin(giulia_recom["Food_code"]))]

print(len(q2))
q2[['Food_code', 'Main_food_description', 'novaclass']] #, 'Predicted Class'
# giulia_recom[giulia_recom["Food_code"].isin(food_items_classified_by_NOVA)]

## Compare with current training data

In [ ]:
# training_df = pd.read_csv('D:/Dropbox (CCNR)/Foodome Team Folder/Ravandi, Babak FDB/FoodomeDev/data/FNDDS/FNDDS_2009-10_Training_data_recom_Giulia_Raw_pythonlabel_mod.csv')

training_df = pd.read_csv('D:\Dropbox (CCNR)\Foodome Team Folder\Menichetti, Giulia\FoodProcessing\Datasets\Training Dataset NOVA\FNDDS_2009-10_Training_data_recom_Giulia_Raw_pythonlabel_mod.csv')

In [ ]:
# NOVA_f = train_cor['Given to Giulia from NOVA Peopl'][train_cor['Given to Giulia from NOVA Peopl']['fc_or_sr'] == 'f']
NOVA_f = train_cor['Given to Giulia from NOVA Peopl']
NOVA_f = NOVA_f.drop_duplicates('dr12ifdcd').reset_index(drop=True)
NOVA_f

cmp_df = pd.merge(
    training_df[['Food_code', 'Main_food_description', 'novaclass']],
    NOVA_f[['dr12ifdcd', 'Main_food_description', 'fc_or_sr', 'FC_nova_group', 'SR_nova_group']],
    left_on='Food_code', right_on='dr12ifdcd')

cmp_df[cmp_df['novaclass'] != cmp_df['FC_nova_group']]

# Check number of covars makes senses
For example 'prostate_cancer_self_report' must have less than 10 covars

In [ ]:
path_reg_analysis = 'D:/Dropbox (CCNR)/Foodome Team Folder/Menichetti, Giulia/FoodProcessing/Datasets/reg_analysis/'

dir_cohort = 'caloric_intake_PSJ1_58_nuts_y234'
cohorts_str = '234'

In [ ]:
df_des = pd.read_csv('{}/{}/design_dataset_{}_cohort{}.csv'.format(
    path_reg_analysis, dir_cohort, dir_cohort, cohorts_str
))

N = len(df_des)

N

In [ ]:
metrics = ['RW.WFDPI.mean.of.both.days.sum',
         'WFDPI.mean.of.both.days.sum',
         'WCDPI.mean.of.both.days.sum',
         'HEI2015_TOTAL_SCORE'][1:2]

adjusting_vars = ['BMXBMI', 'RIDAGEYR', 'female',
         'INDFMPIR', #poverty income ratio
         'white', 'black', 'mexican', 'other_hispanic'
         ,'Total.calories.consumed.mean.both.days']

In [ ]:
pred_var = ['prostate_cancer_self_report'][0]

df_var = df_des[['SEQN', pred_var] + adjusting_vars]

df_var = df_var[~df_var[pred_var].isnull()]

df_var

In [ ]:
print('For var "{}" value counts for columns:\n'.format(pred_var))

pd.DataFrame(df_var.groupby(
    ['female', pred_var]).size()).sort_values(by='female', ascending=False)

In [ ]:
df_var[(df_var['female'] == 1) & (df_var['prostate_cancer_self_report'] == 1)]

## Check if LBXGLU is integer or float so we wont have rounding error!

In [ ]:
path_patel_db = 'D:/Dropbox (CCNR)/Foodome Team Folder/Menichetti, Giulia/FoodProcessing/Datasets/Deisy GLMM/Exposome Patel/nh_99-06/'

df_patel = pd.read_csv(path_patel_db + 'MainTable.csv')

In [ ]:
LBXGLU = df_patel[~df_patel['LBXGLU'].isnull()]['LBXGLU'].sort_values()

LBXGLU

In [ ]:
LBXGLU_val_counts = pd.DataFrame(LBXGLU.value_counts()).reset_index().rename(columns={'index': 'value'}).sort_values('value')

LBXGLU_val_counts[0:500]